In [1]:
import json
import os
import string
import re
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.io.json import json_normalize

import ast
import io

In [2]:
#loading the file
file_path = r"C:\Users\manas\Documents\Master Thesis\Talent_dataset\UM_talent_dataset_1m.json"

In [3]:
#loading line by line of json data because it is in line format
data = [json.loads(line) for line in open(file_path, 'r')]

In [6]:
data[1:2]

[{'id': '65829941-d7ce-4f4e-81da-d027047c2690',
  'location': {'city': 'Hendrik-Ido-Ambacht',
   'country': 'Netherlands',
   'longitude': 4.639505386352539,
   'latitude': 51.8423957824707},
  'educations': [],
  'work_experiences': [{'title': 'gepensioneerd',
    'company_name': 'Geen meer',
    'location': {'city': None,
     'country': None,
     'longitude': None,
     'latitude': None},
    'start_date': None,
    'end_date': None,
    'classification': {'function_type': None,
     'function_type_id': None,
     'function_cat': None,
     'function_cat_id': None,
     'level': None}}],
  'skills': [],
  'languages': [{'language': 'nl', 'proficiency': None}],
  'certificates': [],
  'working_years': None}]

In [5]:
#convert json data into dataframe format
df = pd.DataFrame(data)

In [6]:
#use json_normalize in order to convert each skills columns into dataframe format
exploded = df.explode("skills")
df_exploded = pd.concat([exploded["id"].reset_index(drop=True), pd.json_normalize(exploded["skills"])], axis=1)

In [7]:
df_exploded.head(3)

,id,skill,skill_id,skill_type_id
0,52bfc9d1-2a7a-4880-a6fa-544256633b1f,NaN,NaN,NaN
1,65829941-d7ce-4f4e-81da-d027047c2690,NaN,NaN,NaN
2,9976ce72-93fd-47ec-8fab-5848fbdb800b,EMDR,12473585.0,20950360.0


In [8]:
#collecting the required columns
df_v2=df_exploded[['id','skill','skill_id']]
df_v2.head(10)

,id,skill,skill_id
0,52bfc9d1-2a7a-4880-a6fa-544256633b1f,NaN,NaN
1,65829941-d7ce-4f4e-81da-d027047c2690,NaN,NaN
2,9976ce72-93fd-47ec-8fab-5848fbdb800b,EMDR,12473585.0
3,9976ce72-93fd-47ec-8fab-5848fbdb800b,i.o,955308.0
4,ca6cb7f1-0eb9-424f-99f4-2c0938aebe07,Product Development,972534.0
5,3d86309e-64f6-4df8-ba60-cce431870bfb,business development,972528.0
6,3d86309e-64f6-4df8-ba60-cce431870bfb,Automatisering,1588585.0
7,b755868b-fbab-4245-9541-636f651e7de2,Marketing Communications,972527.0
8,b755868b-fbab-4245-9541-636f651e7de2,Mobile Devices,972539.0
9,b755868b-fbab-4245-9541-636f651e7de2,Research,972565.0


In [9]:
#converting it into string format in order to apply the removing punctuations
df_v3 = df_v2.astype(str)
df_v3

,id,skill,skill_id
0,52bfc9d1-2a7a-4880-a6fa-544256633b1f,nan,nan
1,65829941-d7ce-4f4e-81da-d027047c2690,nan,nan
2,9976ce72-93fd-47ec-8fab-5848fbdb800b,EMDR,12473585.0
3,9976ce72-93fd-47ec-8fab-5848fbdb800b,i.o,955308.0
4,ca6cb7f1-0eb9-424f-99f4-2c0938aebe07,Product Development,972534.0
...,...,...,...
6763203,ecde5f27-c198-469d-994f-22e7e1ac981b,Intensive care,9713217.0
6763204,ecde5f27-c198-469d-994f-22e7e1ac981b,cardiologie,958373.0
6763205,2a7a816f-1218-4f08-a4c8-9c5dac9cd02b,basisonderwijs,7548068.0
6763206,f1c05f61-1076-4896-a8db-9191bc31d50e,D,955691.0


In [10]:
#removing the nan and selecting the non-zero values
data = df_v3[(df_v3.id != 'nan') & (df_v3.skill  != 'nan') & (df_v3.skill_id  != 'nan')  ]
data

,id,skill,skill_id
2,9976ce72-93fd-47ec-8fab-5848fbdb800b,EMDR,12473585.0
3,9976ce72-93fd-47ec-8fab-5848fbdb800b,i.o,955308.0
4,ca6cb7f1-0eb9-424f-99f4-2c0938aebe07,Product Development,972534.0
5,3d86309e-64f6-4df8-ba60-cce431870bfb,business development,972528.0
6,3d86309e-64f6-4df8-ba60-cce431870bfb,Automatisering,1588585.0
...,...,...,...
6763203,ecde5f27-c198-469d-994f-22e7e1ac981b,Intensive care,9713217.0
6763204,ecde5f27-c198-469d-994f-22e7e1ac981b,cardiologie,958373.0
6763205,2a7a816f-1218-4f08-a4c8-9c5dac9cd02b,basisonderwijs,7548068.0
6763206,f1c05f61-1076-4896-a8db-9191bc31d50e,D,955691.0


In [11]:
def spe_cha(col):
    special = re.findall(r'[^\w\s]|_',col)
    lista_special_characters.extend(special)
    remove_special = re.sub('[^A-Za-z0-9]+', '', col)
    return remove_special

In [ ]:
lista_special_characters = []
data['skill'] = data.apply(lambda row: spe_cha(row['skill']), axis=1)

In [14]:
lista_special_characters = list(dict.fromkeys(lista_special_characters))

print(lista_special_characters)

['.', '-', '&', ')', '|', '\\', '/', '(', '"', "'", '+', '#', '`', '@', '!', '?', ',', '=', '>', '*', '~', '<', ']', '[', '%', ':', '$', '{', '_', '^', ';']


In [48]:
#saving the data of id-skillid-skill into csv files
data.to_csv(r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\id_skill-id_skill.csv",index=False,header=True)

In [74]:
#removing extra spaces and string punctuation 
p = re.compile(r'\D')
data['skill'] = data['skill'].str.replace(r'[^\w\s]+', '')
data

C:\Users\manas\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
C:\Users\manas\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,skill,skill_id
2,9976ce72-93fd-47ec-8fab-5848fbdb800b,EMDR,12473585.0
3,9976ce72-93fd-47ec-8fab-5848fbdb800b,io,955308.0
4,ca6cb7f1-0eb9-424f-99f4-2c0938aebe07,Product Development,972534.0
5,3d86309e-64f6-4df8-ba60-cce431870bfb,business development,972528.0
6,3d86309e-64f6-4df8-ba60-cce431870bfb,Automatisering,1588585.0
...,...,...,...
6763203,ecde5f27-c198-469d-994f-22e7e1ac981b,Intensive care,9713217.0
6763204,ecde5f27-c198-469d-994f-22e7e1ac981b,cardiologie,958373.0
6763205,2a7a816f-1218-4f08-a4c8-9c5dac9cd02b,basisonderwijs,7548068.0
6763206,f1c05f61-1076-4896-a8db-9191bc31d50e,D,955691.0


In [62]:
data.to_csv(r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\removed_punctuation.csv",index=False,header=True)

In [75]:
#applying groupby on skill id in order to get group of similar words of skills labels
df_v4 = data.groupby('skill_id').skill.apply(lambda x : ','.join(set(x))).reset_index()
df_v4

,skill_id,skill
0,10124166.0,"advanced planning scheduling,Advanced Planning..."
1,10209981.0,"boudoir photography,boudoir fotografie"
2,10229057.0,Wide Area Network WAN
3,10242030.0,"CiVision,Civision"
4,10309339.0,"PreBreeding,Prebreeding"
...,...,...
5329,977806.0,"wrongful death,Wrongful Death"
5330,977807.0,"Compliance ,Compliance ,wettelijke compliance..."
5331,977882.0,"axapta,Microsoft dynamics AX,AXAPTA,Microsoft ..."
5332,9802196.0,"Sports injuries,sportsinjuries,Sports injury,s..."


In [64]:
df_v4.to_csv(r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\skill_id_group_skill.csv",header=True,index=False)

In [76]:
first_words = df_v4.applymap(lambda x: x.split(',')[0])
print (first_words)

        skill_id                         skill
0     10124166.0  advanced planning scheduling
1     10209981.0           boudoir photography
2     10229057.0         Wide Area Network WAN
3     10242030.0                      CiVision
4     10309339.0                   PreBreeding
...          ...                           ...
5329    977806.0                wrongful death
5330    977807.0                   Compliance 
5331    977882.0                        axapta
5332   9802196.0               Sports injuries
5333   9856113.0                  knee surgery

[5334 rows x 2 columns]


In [66]:
first_words.to_csv(r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\skill_id_only_one_word.csv",header=True,index=False)

In [77]:
df_v4

,skill_id,skill
0,10124166.0,"advanced planning scheduling,Advanced Planning..."
1,10209981.0,"boudoir photography,boudoir fotografie"
2,10229057.0,Wide Area Network WAN
3,10242030.0,"CiVision,Civision"
4,10309339.0,"PreBreeding,Prebreeding"
...,...,...
5329,977806.0,"wrongful death,Wrongful Death"
5330,977807.0,"Compliance ,Compliance ,wettelijke compliance..."
5331,977882.0,"axapta,Microsoft dynamics AX,AXAPTA,Microsoft ..."
5332,9802196.0,"Sports injuries,sportsinjuries,Sports injury,s..."
